In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *
from xfem.lsetcurv import *
from ngsolve.webgui import Draw

In [2]:
"""
from ngsolve import *
from math import pi

maxh = 0.1  # Maximaler Elementabstand
# Analytische Geschwindigkeit (als GridFunction oder CoefficientFunction)
uexact = CoefficientFunction((
    sin(pi*x) * cos(pi*y),   # u(x,y)
    -cos(pi*x) * sin(pi*y)   # v(x,y)
))
pexact = sin(pi*x) * sin(pi*y)
# Laplace von uexact (Komponentenweise)
lapu_x = -pi**2 * sin(pi*x) * cos(pi*y) * 2
lapu_y = -pi**2 * (-cos(pi*x) * sin(pi*y)) * 2  # Minuszeichen kommt von v

# Gradient von p
dp_dx = pi * cos(pi*x) * sin(pi*y)
dp_dy = pi * sin(pi*x) * cos(pi*y)

# Gesamte rechte Seite f = -Δu + ∇p
f = CoefficientFunction((
    -lapu_x + dp_dx,
    -lapu_y + dp_dy
))

r = sqrt(x**2 + y**2)
levelset = r - 1
#f= CF((0,x))

#uexact = CoefficientFunction((    -r * (1 - r**2) * y, r * (1 - r**2) * x))

# Analytischer Druck
#pexact = (r + 3 - 15 * r**2) * y
u1 = -4*y * (1 - x**2 - y**2)
u2 = 4*x*(1 - x**2 - y**2)
uexact = CoefficientFunction((u1, u2))

pexact = sin(x)*cos(y)

# Laplace-Anteil
lapu1 = 32*y
lapu2 = -32*x


# Gradient von p
dpdx = cos(x) * cos(y)
dpdy = -sin(x) * sin(y)

f = CoefficientFunction((
    -lapu1 - dpdx,
    -lapu2 - dpdy
))

from ngsolve.webgui import Draw

order = 2

square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)

lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order, threshold=0.1,discontinuous_qn=True)# Higher order level set approximation 
deformation = lsetmeshadap.CalcDeformation(levelset)

lsetp1 = GridFunction(H1(mesh,order=1,autoupdate=True),autoupdate=True)
InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
#Draw(lsetp1, mesh, f"levelset_maxh{maxh}")
Draw(lsetp1,uexact, CF((0,0)), mesh, "uexact")
"""

'\nfrom ngsolve import *\nfrom math import pi\n\nmaxh = 0.1  # Maximaler Elementabstand\n# Analytische Geschwindigkeit (als GridFunction oder CoefficientFunction)\nuexact = CoefficientFunction((\n    sin(pi*x) * cos(pi*y),   # u(x,y)\n    -cos(pi*x) * sin(pi*y)   # v(x,y)\n))\npexact = sin(pi*x) * sin(pi*y)\n# Laplace von uexact (Komponentenweise)\nlapu_x = -pi**2 * sin(pi*x) * cos(pi*y) * 2\nlapu_y = -pi**2 * (-cos(pi*x) * sin(pi*y)) * 2  # Minuszeichen kommt von v\n\n# Gradient von p\ndp_dx = pi * cos(pi*x) * sin(pi*y)\ndp_dy = pi * sin(pi*x) * cos(pi*y)\n\n# Gesamte rechte Seite f = -Δu + ∇p\nf = CoefficientFunction((\n    -lapu_x + dp_dx,\n    -lapu_y + dp_dy\n))\n\nr = sqrt(x**2 + y**2)\nlevelset = r - 1\n#f= CF((0,x))\n\n#uexact = CoefficientFunction((    -r * (1 - r**2) * y, r * (1 - r**2) * x))\n\n# Analytischer Druck\n#pexact = (r + 3 - 15 * r**2) * y\nu1 = -4*y * (1 - x**2 - y**2)\nu2 = 4*x*(1 - x**2 - y**2)\nuexact = CoefficientFunction((u1, u2))\n\npexact = sin(x)*cos(y)\n\

In [3]:
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=0.2)
mesh = Mesh(ngmesh)

# possible sol for square

from ngsolve import IfPos

def ngsmax(a, b):
    return IfPos(a - b, a, b)
levelset= ngsmax(x**2,y**2)- 1 
  
# Manufactured solution
uexact = CoefficientFunction((
    sin(pi*x) * cos(pi*y),
    -cos(pi*x) * sin(pi*y)
))
pexact = sin(pi*x) * sin(pi*y)

lapu_x = -pi**2 * sin(pi*x) * cos(pi*y) * 2
lapu_y = -pi**2 * (-cos(pi*x) * sin(pi*y)) * 2
dp_dx = pi * cos(pi*x) * sin(pi*y)
dp_dy = pi * sin(pi*x) * cos(pi*y)
f = CoefficientFunction((
    -lapu_x - dp_dx,
    -lapu_y - dp_dy
))
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=1, threshold=0.1,discontinuous_qn=True)# Higher order level set approximation 
deformation = lsetmeshadap.CalcDeformation(levelset)
lsetp1 = lsetmeshadap.lset_p1
lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:


In [ ]:
from ngsolve.webgui import Draw

errors = []
hs = []
orders = []

for maxh in [0.1,0.05,0.025]:
    print(maxh)
    for order in [3]:
        # Stabilization parameter for ghost-penalty
        gamma_stab = 100
        # Stabilization parameter for Nitsche
        lambda_nitsche = 10 * order * order

        beta0 =10 * order**2
        beta1 = 1
        beta2 = 10 * order**2
        beta3 = 1
        square = SplineGeometry()
        square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
        ngmesh = square.GenerateMesh(maxh=maxh)
        mesh = Mesh(ngmesh)
        #lsetp1.Set(levelset)

        lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order, threshold=0.1,discontinuous_qn=True)# Higher order level set approximation 
        deformation = lsetmeshadap.CalcDeformation(levelset)

        lsetp1 = GridFunction(H1(mesh,order=1,autoupdate=True),autoupdate=True)
        InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
        #Draw(lsetp1, mesh, f"levelset_maxh{maxh}")
        #DrawDC(lsetp1,uexact, CF((0,0)), mesh, "uexact")
        ci = CutInfo(mesh, lsetp1)
        hasneg = ci.GetElementsOfType(HASNEG)
        neg = ci.GetElementsOfType(NEG)
        hasif = ci.GetElementsOfType(IF)
        haspos = ci.GetElementsOfType(HASPOS)
        ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
        patch_elements = ci.GetElementsOfType(ANY)
        #cut_elements = ci.GetElementsOfType(HASIF)  


        interior_facets = GetFacetsWithNeighborTypes(mesh, a=neg, b=neg)
        interface_facet_set = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasneg)
        stable_elem =GetFacetsWithNeighborTypes(mesh,a = hasif, b= patch_elements)

        
        h = specialcf.mesh_size
        n = Normalize(grad(lsetp1))
        nF= specialcf.normal(mesh.dim)

        # integration domains:
        dX = dCut(lsetp1, NEG, definedonelements=hasneg,deformation=deformation)
        ds = dCut(lsetp1, IF, definedonelements=hasif,deformation=deformation)

        dw = dFacetPatch(definedonelements=ba_facets, deformation=deformation)
        dw_int = dFacetPatch(definedonelements=interior_facets,deformation=deformation)
        dw_interface = dFacetPatch(definedonelements=interface_facet_set, deformation=deformation)
        
        GP_facets = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasneg, use_and=True)
        hasneg_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg, use_and=True)
        test_elem =GetElementsWithNeighborFacets(mesh,GP_facets)
        #def get_surrounding_elements(elements,iter=1 ):
        #    for i in range(2*iter):
        #        ba_facets = GetFacetsWithNeighborTypes(mesh, a=elements, b=elements, use_and=False, bnd_val_a= False,bnd_val_b=False)

        #        ci_elements = GetElementsWithNeighborFacets(mesh, ba_facets)
        #    return ci_elements

        #Draw(BitArrayCF(get_surrounding_elements(test_elem)),mesh)
        d_stable_GP = dx(definedonelements = test_elem, deformation = deformation,skeleton = True)
        ds_inner_facets = dCut(lsetp1, NEG, definedonelements=hasneg_facets,deformation=deformation, skeleton=True)
        
        #ds_stable_facets = dx(definedonelements = stable_elem, deformation = deformation, skeleton = True)
        V = VectorH1(mesh, order=order,dgjumps=True)
        V = Compress(V, GetDofsOfElements(V,hasneg))
        Q = L2(mesh, order=order-1)
        Q = Compress(Q, GetDofsOfElements(Q,hasneg))
        Z = NumberSpace(mesh)
        X = V*Q*Z
        (u,p,z),(v,q,y) = X.TnT()
        gfu = GridFunction(X)

        def jump(f):
            return f - f.Other()
        def jumpn(f):
            return Grad(f)*n - Grad(f).Other()*n
        def jumpnF(f):
            return Grad(f)*nF - Grad(f).Other()*nF

        a = BilinearForm(X)
        stokes = InnerProduct(Grad(u), Grad(v))*dX - div(u)*q*dX - div(v)*p*dX
        a += stokes
        a += -(Grad(u)*n * v + Grad(v)*n * u) * ds + gamma_stab / h * u * v * ds #nitzshe stabilization
        a += (q*n * u + p*n * v) * ds
        a += p*y *dX + q *z*dX
        a += beta2*h*jumpnF(u) * jumpnF(v) * d_stable_GP #velocity ghost penalty
        a += -beta0*h * jump(p) * jump(q) * d_stable_GP #pressure ghost penalty
        a += beta0* jump(p)*jump(q) * ds_inner_facets #c-term
        #a += beta2* InnerProduct(jump(Grad(u)), jump(Grad(v))) * dw_interface #velocity ghost penalty
        #a += -beta0 * InnerProduct(jump(p), jump(q)) * dw_interface #pressure ghost penalty

        #a += beta2* InnerProduct(jump(Grad(u)), jump(Grad(v))) * dw_interface
        a.Assemble()

        rhs = LinearForm(X)  # oder f*v*dx mit f gegeben
        rhs += InnerProduct(f, v)* dX
        rhs += -(Grad(v)*n * uexact) * ds + gamma_stab / h * uexact * v * ds+q*n*uexact *ds
        rhs.Assemble()
        gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * rhs.vec
        
        error_u = sqrt(Integrate( (gfu.components[0] - uexact) ** 2*dX, mesh ))
        print("||u_h - u_exact||_L2 =", error_u)
        #error_p = sqrt(Integrate( (gfu.components[1] - pexact) ** 2*dx, mesh ))
        #print("||p_h - p_exact||_L2 =", error_p ,"order=", order, "maxh=", maxh)

        #l2error.append(sqrt(Integrate((gfu - exact)**2*dx, mesh)))
        #Draw(gfu.components[0], mesh, "u")
        #DrawDC(lsetp1,gfu.components[0], CF((0,0)), mesh, "u")
        #Draw(gfu.components[1], mesh, "p")

        DrawDC(lsetp1,gfu.components[0], CF((0,0)), mesh, "u")
        #DrawDC(lsetp1,gfu.components[1], 0, mesh, "p")
        errors.append(error_u)
        hs.append(maxh)
        orders.append(order)
        



0.1
||u_h - u_exact||_L2 = 0.02896358955036486
webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.05
||u_h - u_exact||_L2 = 6.361737922408173e-05
webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.035
||u_h - u_exact||_L2 = 1.0457295735121026e-06
webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…